In [1]:
from pathlib import Path

import pandas as pd
from pandas.errors import EmptyDataError 

import matplotlib.pyplot as plt

from datetime import datetime as dt
from datetime import time, timedelta

In [2]:
# This method takes in paramaters:
# 1) recover_folder: Name of the folder where the detections are stored.
# 2) AUDIO_DUR: The AudioMoth's configuration for the length of each recording.

# This method returns:
# - A DataFrame object that has columns:
#   - File Names, Date, Start Time, End Time, # of LF detections, # of HF detections
#   - File Names are strings and # of LF/HF detections are integers.
#   - Times are in UTC format and Dates are datetime.datetime objects.

def generate_df(recover_folder, audio_dur=pd.DateOffset(minutes=29, seconds=55)):

    # detection_dir is the recover-DATE-UNIT_NUM-detect folder where our detections are.
    detection_dir = f"detections/{recover_folder}"

    # Construct path object linked to the directory of files for datetime-parsing
    file_dir = Path(detection_dir)
    # We use this to only extract the original file names of the recordings.
    # The detection files will be assembled below.
    sorted_files = sorted(file_dir.glob('hf_*.txt'))

    # Create empty DataFrame object with all the required columns    
    df = pd.DataFrame(columns=["File Names", "Date", "Start Time (UTC)",
                       "End Time (UTC)", "# of LF detections", "# of HF detections"])
    
    # Iterate through all file paths to extract and store table information for each file
    for i, file in enumerate(sorted_files):
        # Extract name of each file as it is
        file_name = file.name
        # Extracting the datetime object from the name of each file
        file_info = dt.strptime(file_name, "hf_%Y%m%d_%H%M%S.WAV.txt")

        # Extract recording name, date, start time, and end time for each file
        recording_name = file_info.strftime("%Y%m%d_%H%M%S.WAV")
        date = file_info.date()
        s_time = file_info.time()
        e_time = (file_info+audio_dur).time()

        # Calling read_detection to return the table of selections as a dataframe
        # The detections appear twice: in waveform view and spectrogram view, 
        # so we half the total number of detections
        lf_file_detections = read_detection(detection_dir, recording_name, "lf")
        num_lf_detections = lf_file_detections.shape[0]/2
        hf_file_detections = read_detection(detection_dir, recording_name, 'hf')
        num_hf_detections = hf_file_detections.shape[0]/2
        
        # Add new row with the extracted information
        df.loc[len(df.index)] = [recording_name, date, s_time, e_time, num_lf_detections, num_hf_detections]
        
    # This section checks if the last date has information from 0:00 to 24:00
    # If it doesn't we pad the date with time information so our plots can be comparable
    # The # of LF/HF detections will be NaN in the dataframe.
    while (s_time != time(23, 30)):
        date = df.loc[len(df.index)-1]["Date"]
        s_time = (dt.combine(date, s_time)+pd.DateOffset(minutes=30)).time()
        file_info = dt.combine(date, s_time)
        recording_name = file_info.strftime("%Y%m%d_%H%M%S.WAV")
        e_time = (file_info+audio_dur).time()
        df.loc[len(df.index)] = [recording_name, date, s_time, e_time, None, None]
    
    return df

In [3]:
# Given parameters:
# 1) detection_dir is the recover-DATE-UNIT_NUM-detect folder
# 2) recording_name is the name of our recording: DATE_TIME.WAV
# 4) det_type can either be 'lf' or 'hf'

# Output:
# The table of detections following the format of RavenPro

def read_detection(detection_dir, recording_name, det_type):
    
    file_name = f"{det_type}_{recording_name}.txt"
    file_path = f"{detection_dir}/{file_name}"
    
    if (Path(file_path).is_file()):
        try:
            df_detection = pd.read_csv(file_path, sep='\t')
        except EmptyDataError:
            print(f"{file_path} is empty")
            
    return df_detection

In [73]:
def plot_separate(df, save=True):
    
    # To plot each day's activity separately, group by rows that have the same date
    # We need a list of unique dates from our detection files
    unique_dates = df["Date"].unique()

    s_times = []
    i=0
    s_times.append(time(0, 0, 0))
    while (s_times[i] != time(hour=23, minute=30, second=0)):
        old_dt = (dt.combine(unique_dates[0], s_times[i]))
        new_dt = old_dt+pd.DateOffset(minutes=30)
        s_times.append(new_dt.time())
        i = i+1
        
    print(s_times)
    
    for date in unique_dates:
        day_df = df.loc[df['Date'] == date]
        
        fig = day_df.plot.bar(x="Start Time (UTC)", figsize=(12, 4), fontsize=12, rot=45)
        fig.set_title(date, fontsize=14)
        
        if save:
            save_folder = f"../results/raven_energy_detector_raw/call_num_summary/{recover_folder}/FIGS"
            save_dir = Path(save_folder)
            save_dir.mkdir(parents=True, exist_ok=True)
            save_path = Path(f"{save_folder}/{date}.png")
            fig.get_figure().savefig(save_path, facecolor='w')
            
        fig.set_xticks(fig.get_xticks()[::2])        

In [74]:
def plot_total(df):
    
    fig = df.plot.bar(x="Start Time (UTC)", figsize=(12, 4), fontsize=12, rot=45)
    fig.set_title("Total Deployment Session", fontsize=14)
    fig.set_xticks(fig.get_xticks()[::4])

In [75]:
# These are constants that we absolutely need to know before running any scripts

# Constant DATE is the date of our recovered data that we are extracting detections from
DATE = "20220725"
# Constant SD_CARD_NUM is the SD CARD # that had been deployed for that session
SD_CARD_NUM = "010"

In [76]:
recover_folder = f"recover-{DATE}-{SD_CARD_NUM}-detect"
df = generate_df(recover_folder)

In [77]:
plot_separate(df)

[datetime.time(0, 0), datetime.time(0, 30), datetime.time(1, 0), datetime.time(1, 30), datetime.time(2, 0), datetime.time(2, 30), datetime.time(3, 0), datetime.time(3, 30), datetime.time(4, 0), datetime.time(4, 30), datetime.time(5, 0), datetime.time(5, 30), datetime.time(6, 0), datetime.time(6, 30), datetime.time(7, 0), datetime.time(7, 30), datetime.time(8, 0), datetime.time(8, 30), datetime.time(9, 0), datetime.time(9, 30), datetime.time(10, 0), datetime.time(10, 30), datetime.time(11, 0), datetime.time(11, 30), datetime.time(12, 0), datetime.time(12, 30), datetime.time(13, 0), datetime.time(13, 30), datetime.time(14, 0), datetime.time(14, 30), datetime.time(15, 0), datetime.time(15, 30), datetime.time(16, 0), datetime.time(16, 30), datetime.time(17, 0), datetime.time(17, 30), datetime.time(18, 0), datetime.time(18, 30), datetime.time(19, 0), datetime.time(19, 30), datetime.time(20, 0), datetime.time(20, 30), datetime.time(21, 0), datetime.time(21, 30), datetime.time(22, 0), datetim

KeyError: "None of [Index([00:00:00, 00:30:00, 01:00:00, 01:30:00, 02:00:00, 02:30:00, 03:00:00,\n       03:30:00, 04:00:00, 04:30:00, 05:00:00, 05:30:00, 06:00:00, 06:30:00,\n       07:00:00, 07:30:00, 08:00:00, 08:30:00, 09:00:00, 09:30:00, 10:00:00,\n       10:30:00, 11:00:00, 11:30:00, 12:00:00, 12:30:00, 13:00:00, 13:30:00,\n       14:00:00, 14:30:00, 15:00:00, 15:30:00, 16:00:00, 16:30:00, 17:00:00,\n       17:30:00, 18:00:00, 18:30:00, 19:00:00, 19:30:00, 20:00:00, 20:30:00,\n       21:00:00, 21:30:00, 22:00:00, 22:30:00, 23:00:00, 23:30:00],\n      dtype='object')] are in the [columns]"

In [ ]:
plot_total(df)